## Loading libs

In [12]:
from .utils import files
from
import pprint

# Factor plot
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
import seaborn as sns
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 10]
%matplotlib inline



def get_avg_losses():
    files, filenames = unique('final/*/*.csv')

    # Iterate over unique filenames; read CSVs, concat DFs
    avg_losses = pd.DataFrame()
    losses =pd.DataFrame()
    for fn in filenames:
       # fn = files['filename'].unique()[1]
        name = fn.split('.')[0]
        print(name)

        dir, f = split(fn)
        f = f.split('_')
        if any(x in fn for x in ['default', 'ompl']):
            mode = f[-1].split('.')[-2]
        else:
            planner_fn = f[-1].split('.')[-2]
            mode = f[2]  

        paths = files[files['filename'] == fn]['fullpath'] # Get list of fullpaths from unique filenames
        dfs = []
        
        losses=pd.DataFrame()
        for idx, path in enumerate(paths):
            if not any(x in path for x in ['default', 'ompl']):
                df = pd.read_csv(path, index_col=False, sep=',').dropna(axis=1)
                df_resample = resample(df, '2S')
                df_resample = df_resample['loss']     
                dfs.append(df_resample) 
            else:
                df = pd.read_csv(path, index_col=False, sep=',').dropna(axis=1)
                df_resample = resample(df, '2S')
                df_resample = df_resample['t_avg_plan_time']     
                dfs.append(df_resample) # Get list of dataframes from CSV file paths

            losses = pd.concat([losses, pd.DataFrame({name+str(idx) :df_resample})], axis=1)

        avg_losses = pd.concat([avg_losses, pd.DataFrame({name :losses.mean(axis=1)})], axis=1)
    return avg_losses

def factor_plot(fps, planner_select, planner, mode_select=None):
    fps = sorted(fps.copy())
    plt.rcParams['figure.figsize'] = [15, 10]
    fig,ax = plt.subplots(1)
    cm = plt.get_cmap('jet')
    ax.set_prop_cycle('color', [cm(i) for i in np.linspace(0, 1, 10)])
    dir, fn = split(fps[0])
    
    elapsed_time = np.linspace(0, 7200, 500)
    default_fn = planner_select+'_default.csv'
    default_df = pd.read_csv(join(dir, default_fn), index_col=False)
    default = float(default_df[(default_df['planner'].str.contains(planner))]['t_avg_plan_time'])
    default_df = pd.DataFrame(
        {'elapsed_time': elapsed_time, 'default': default})
    default_df['default'] = default/default_df['default']
    ax.plot('elapsed_time', 'default', data=default_df, label='default')

    ompl_fn = planner_select+'_ompl.csv'
    ompl_df = pd.read_csv(join(dir, ompl_fn), index_col=False)
    ompl = float(ompl_df[(ompl_df['planner'].str.contains(planner))]['t_avg_plan_time'])
    ompl_df = pd.DataFrame({'elapsed_time': elapsed_time, 'ompl': ompl})
    ompl_df['ompl'] = default/ompl_df['ompl']
    ax.plot('elapsed_time', 'ompl', data=ompl_df, label='ompl')
    
    
    # Planner select filepaths
    for fp in fps:
        if planner_select in fp:
            if not any(x in fp for x in ['default', 'ompl']):
                # f = <planner_select>_<mode>_<planner>.csv
                dir, fn = split(fp)
                fn = fn.split('_')
                planner_fn = fn[-1].split('.')[-2]
                mode = fn[2]
                
                if (mode_select != None) and (mode not in mode_select):
                    fps.remove(fp)
                else:
                    # Filtering fns for selected planner
                    if re.search(planner_fn, planner, re.IGNORECASE): 
#                         print(fp)
                        mode_df = pd.read_csv(fp, index_col=False)
                        # Remove everything slower than ompl baseline
                        # mode_df.drop(mode_df[mode_df.loss >= ompl].index, inplace=True)
                        max_speed_up = default
                        speed_up = list(mode_df['loss'])
                        for idx, loss in enumerate(speed_up):
                            if loss < max_speed_up:
                                max_speed_up = loss
                            else:
                                speed_up[idx] = max_speed_up
                                                                   
                        mode_df = pd.DataFrame({'elapsed_time': mode_df['elapsed_time'], mode: speed_up})
                        mode_df[mode+'_factor'] = default/mode_df[mode]
                        ax.plot('elapsed_time', mode+'_factor', data=mode_df, label=mode+'_factor')

    
    plt.title(planner_select + ' ' + planner)
    plt.xticks([0, 1440, 2880, 4320, 5760, 7200])
    #                     plt.yticks([0.8, 1.6, 2.4, 3.1, 3.9, 4.7])
    plt.xlabel('Time spent exploring solutions (secs)')
    plt.ylabel('Speed up over default (secs)')
    handles,labels = ax.get_legend_handles_labels()
    ax.legend(handles,labels,loc='upper right')
    plt.show()

def avg_factor_plot(avg_losses, planner_select, planner, benchmark='default', mode_select=None):
    plt.rcParams['figure.figsize'] = [15, 10]
    df = avg_losses.copy()
    def_cols = [planner_select+'_default', planner_select+'_ompl']
    cols = [c for c in df.columns if planner_select in c if planner in c]
    cols = sorted(cols)
    cols = def_cols + cols

    ps_df = df[cols]
    legend = [c.split('_')[2] for c in cols]
    ps_df.columns = legend

    bm = float(ps_df['default'][0])
    if benchmark == 'ompl':
        bm = float(ps_df['ompl'][0])
        
    speedup_df = pd.DataFrame({'default': bm/ps_df['default'], 'ompl': bm/ps_df['ompl']})
    
    modes = legend
    if (mode_select != None):
        modes = ['default', 'ompl']
        modes = modes+ mode_select

    for mode in modes:
        if not any(x in mode for x in ['default', 'ompl']): 
            speedup = list(ps_df[mode])
            max_speedup = bm
            for idx, loss in enumerate(speedup):
                if loss < max_speedup:
                    max_speedup = loss
                else:
                    speedup[idx] = max_speedup

            mode_df = pd.Series(speedup, index=ps_df[mode].index, name=mode)
            mode_df = bm/mode_df
            speedup_df = pd.concat([speedup_df, mode_df], axis=1)
    
#     xtick_vals = [0, 1440, 2880, 4320, 5760, 7200] 
#     xtick_vals =  pd.to_timedelta([datetime.timedelta(seconds=x) for x in xtick_vals])
#     xtick_vals = [strdatetime.timedelta(seconds=x)) for x in xtick_vals]
# #     print(xticks_vals)
    ax = speedup_df.plot(title=planner_select+' '+planner, colormap='jet')
    
    ax.set_xlabel('Time spent exploring solutions (secs)')
    ax.set_ylabel('Speed up over default (secs)')
    ax.legend(loc='upper right')


#     ax.set_xticks(xticks_vals)
#     ax.set_xticklabels(str(xticks_vals))

ModuleNotFoundError: No module named '__main__.utils'; '__main__' is not a package

In [8]:
avg_losses = get_avg_losses()
avg_losses.head()

TypeError: 'bool' object is not callable

## Speedup Factor Plots

In [ ]:
results_fps_1 = [file for file in glob.glob(join('final', 'run_1', '*.csv'))]
print(len(results_fps_1))

results_fps_2 = [file for file in glob.glob(join('final', 'run_2', '*.csv'))]
print(len(results_fps_2))

results_fps_3 = [file for file in glob.glob(join('final', 'run_3', '*.csv'))]
print(len(results_fps_3))

### Cano_etal BKPIECE

In [ ]:
avg_factor_plot(avg_losses, 'Cano_etal', 'BKPIECE' )

In [ ]:
# factor_plot(results_fps_1, 'Cano_etal', 'BKPIECE')
# factor_plot(results_fps_2, 'Cano_etal', 'BKPIECE')
# factor_plot(results_fps_3, 'Cano_etal', 'BKPIECE')

### Cano_etal RRTConnect


In [ ]:
avg_factor_plot(avg_losses, 'Cano_etal', 'RRTConnect')

In [ ]:
# factor_plot(results_fps_1, 'Cano_etal', 'RRTConnect')
# factor_plot(results_fps_2, 'Cano_etal', 'RRTConnect')
# factor_plot(results_fps_3, 'Cano_etal', 'RRTConnect')

### Burger_etal BKPIECE

In [ ]:
avg_factor_plot(avg_losses, 'Burger_etal', 'BKPIECE' ,benchmark='ompl' )

In [ ]:
# factor_plot(results_fps_1, 'Burger_etal', 'BKPIECE')
# factor_plot(results_fps_2, 'Burger_etal', 'BKPIECE')
# factor_plot(results_fps_3, 'Burger_etal', 'BKPIECE')

### Burger_etal RRTConnect

In [ ]:
avg_factor_plot(avg_losses, 'Burger_etal', 'RRTConnect')

In [ ]:
# factor_plot(results_fps_1, 'Burger_etal', 'RRTConnect')
# factor_plot(results_fps_2, 'Burger_etal', 'RRTConnect')
# factor_plot(results_fps_3, 'Burger_etal', 'RRTConnect')

### Burger_etal BiTRRT

In [ ]:
avg_factor_plot(avg_losses, 'Burger_etal', 'BiTRRT')

In [ ]:
# factor_plot(results_fps_1, 'Burger_etal', 'BiTRRT')
# factor_plot(results_fps_2, 'Burger_etal', 'BiTRRT')
# factor_plot(results_fps_3, 'Burger_etal', 'BiTRRT')